In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd

pd.set_option("max_row", 1500)

<IPython.core.display.Javascript object>

In [3]:
import numpy as np

<IPython.core.display.Javascript object>

In [4]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

<IPython.core.display.Javascript object>

In [5]:
noun_verb_raw = pd.read_csv("./Noun Verb Pair Data/noun_verb_processed_data.csv")

<IPython.core.display.Javascript object>

In [6]:
noun_verb_raw

,sentence#,word,pos,tag
0,0.0,Preheat,VB,U-Action
1,0.0,oven,NN,U-Utensil
2,0.0,to,IN,O
3,0.0,425,CD,O
4,0.0,degrees,NNS,O
...,...,...,...,...
6804,507.0,an,DT,NaN
6805,507.0,hour,NN,NaN
6806,507.0,at,IN,NaN
6807,507.0,least,JJS,NaN


<IPython.core.display.Javascript object>

In [7]:
noun_verb_raw = noun_verb_raw.drop(["pos"], axis=1)

<IPython.core.display.Javascript object>

In [8]:
noun_verb_raw

,sentence#,word,tag
0,0.0,Preheat,U-Action
1,0.0,oven,U-Utensil
2,0.0,to,O
3,0.0,425,O
4,0.0,degrees,O
...,...,...,...
6804,507.0,an,NaN
6805,507.0,hour,NaN
6806,507.0,at,NaN
6807,507.0,least,NaN


<IPython.core.display.Javascript object>

In [9]:
# convert all BILU-Ingredients into Ingredients and combine the words into one
noun_verb_raw["tag"].value_counts()

O               2498
U-Action         366
U-Ingredient     241
B-Ingredient      70
L-Ingredient      70
B-Utensil         60
U-Utensil         59
L-Utensil         59
I-Utensil         15
I-Ingredient       3
L-Action           2
B-Action           2
Name: tag, dtype: int64

<IPython.core.display.Javascript object>

In [10]:
noun_verb_raw_consolidated = pd.DataFrame({"sentence#": [], "word": [], "tag": []})
ingredient_word = ""
action_word = ""

for i in range(len(noun_verb_raw)):

    # U-Ingredient
    if noun_verb_raw.iloc[i]["tag"] == "U-Ingredient":
        noun_verb_raw_consolidated = noun_verb_raw_consolidated.append(
            pd.DataFrame(
                {
                    "sentence#": [noun_verb_raw.iloc[i]["sentence#"]],
                    "word": [noun_verb_raw.iloc[i]["word"]],
                    "tag": ["Ingredient"],
                }
            )
        )

    # B-Ingredient
    if noun_verb_raw.iloc[i]["tag"] == "B-Ingredient":
        ingredient_word = noun_verb_raw.iloc[i]["word"]

    # I-Ingredient, will always be followed by B-Ingredient
    if noun_verb_raw.iloc[i]["tag"] == "I-Ingredient":
        ingredient_word = ingredient_word + " " + noun_verb_raw.iloc[i]["word"]

    # L-Ingredient, will always be followed by B-Ingredient
    if noun_verb_raw.iloc[i]["tag"] == "L-Ingredient":
        ingredient_word = ingredient_word + " " + noun_verb_raw.iloc[i]["word"]
        noun_verb_raw_consolidated = noun_verb_raw_consolidated.append(
            pd.DataFrame(
                {
                    "sentence#": [noun_verb_raw.iloc[i]["sentence#"]],
                    "word": [ingredient_word],
                    "tag": ["Ingredient"],
                }
            )
        )

        # reset ingredient_word
        ingredient_word = ""

    # U-Action
    if noun_verb_raw.iloc[i]["tag"] == "U-Action":
        noun_verb_raw_consolidated = noun_verb_raw_consolidated.append(
            pd.DataFrame(
                {
                    "sentence#": [noun_verb_raw.iloc[i]["sentence#"]],
                    "word": [noun_verb_raw.iloc[i]["word"]],
                    "tag": ["Action"],
                }
            )
        )

    # B-Action
    if noun_verb_raw.iloc[i]["tag"] == "B-Action":
        action_word = noun_verb_raw.iloc[i]["word"]

    # I-Ingredient, will always be followed by B-Ingredient
    if noun_verb_raw.iloc[i]["tag"] == "I-Action":
        action_word = action_word + " " + noun_verb_raw.iloc[i]["word"]

    # L-Ingredient, will always be followed by B-Ingredient
    if noun_verb_raw.iloc[i]["tag"] == "L-Action":
        action_word = action_word + " " + noun_verb_raw.iloc[i]["word"]
        noun_verb_raw_consolidated = noun_verb_raw_consolidated.append(
            pd.DataFrame(
                {
                    "sentence#": [noun_verb_raw.iloc[i]["sentence#"]],
                    "word": [action_word],
                    "tag": ["Action"],
                }
            )
        )

        # reset action_word
        action_word = ""


noun_verb_raw_consolidated = noun_verb_raw_consolidated.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [11]:
# lemmatize word column, lowercase word column
noun_verb_raw_consolidated["word"] = noun_verb_raw_consolidated["word"].apply(
    lambda x: (lemmatizer.lemmatize(x)).lower()
)

<IPython.core.display.Javascript object>

In [12]:
# whatever is the ingredient find -1(recent) Action and form association
rules = pd.DataFrame({"Ingredient": [], "Action": []})

for sentence_number, group_df in noun_verb_raw_consolidated.groupby("sentence#"):

    associated_action = ""

    # iterate through each row in group
    for i in range(len(group_df)):

        # keep track of the last Action seen
        if group_df.iloc[i]["tag"] == "Action":
            associated_action = group_df.iloc[i]["word"]

        # once we find an ingredient, then create (ingredient, action) pair
        elif group_df.iloc[i]["tag"] == "Ingredient":
            # if action not found then dont create pair
            if associated_action != "":
                rules = rules.append(
                    pd.DataFrame(
                        {
                            "Ingredient": [group_df.iloc[i]["word"]],
                            "Action": [associated_action],
                        }
                    )
                )
rules.reset_index(drop=True, inplace=True)

<IPython.core.display.Javascript object>

In [13]:
rules

,Ingredient,Action
0,sausage,cut
1,egg,whisk
2,milk,whisk
3,sausage,spoon
4,cheese,sprinkle
5,egg,pour
6,sausage,pour
7,cheese,pour
8,egg,bake
9,ground beef,brown


<IPython.core.display.Javascript object>

# Noun Verb Pair Analysis

In [14]:
rules["Ingredient Count"] = rules.groupby(["Ingredient"]).transform(np.size)

<IPython.core.display.Javascript object>

In [15]:
rules

,Ingredient,Action,Ingredient Count
0,sausage,cut,3
1,egg,whisk,9
2,milk,whisk,2
3,sausage,spoon,3
4,cheese,sprinkle,5
5,egg,pour,9
6,sausage,pour,3
7,cheese,pour,5
8,egg,bake,9
9,ground beef,brown,3


<IPython.core.display.Javascript object>

In [16]:
rules["Ingredient Action Count"] = rules.groupby(["Ingredient", "Action"]).transform(
    np.size
)

<IPython.core.display.Javascript object>

In [17]:
rules["Confidence Percentage"] = (
    np.round(rules["Ingredient Action Count"] / rules["Ingredient Count"], 2) * 100
)

<IPython.core.display.Javascript object>

In [18]:
rules

,Ingredient,Action,Ingredient Count,Ingredient Action Count,Confidence Percentage
0,sausage,cut,3,1,33.0
1,egg,whisk,9,1,11.0
2,milk,whisk,2,1,50.0
3,sausage,spoon,3,1,33.0
4,cheese,sprinkle,5,1,20.0
5,egg,pour,9,2,22.0
6,sausage,pour,3,1,33.0
7,cheese,pour,5,1,20.0
8,egg,bake,9,1,11.0
9,ground beef,brown,3,2,67.0


<IPython.core.display.Javascript object>

In [19]:
results = rules.copy(deep=True)

<IPython.core.display.Javascript object>

In [20]:
results.drop(["Ingredient Count", "Ingredient Action Count"], axis=1, inplace=True)

<IPython.core.display.Javascript object>

In [21]:
# remove duplicate rows
results.drop_duplicates(inplace=True)

# reset the index
results.reset_index(drop=True, inplace=True)

<IPython.core.display.Javascript object>

In [22]:
# create dataframe containing top 5 results
present = pd.DataFrame()

for index, group in results.groupby("Ingredient"):
    present = present.append(
        # top 5
        group.sort_values("Confidence Percentage", ascending=False).head(5)
    )

present = present.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [23]:
present

,Ingredient,Action,Confidence Percentage
0,apple juice,saute,100.0
1,applewood chips,us,100.0
2,avocado,serve,100.0
3,baking powder,mix,100.0
4,banana,top,25.0
5,banana,add,25.0
6,banana,beat,25.0
7,banana,fold,25.0
8,basil,add,100.0
9,bean,add,60.0


<IPython.core.display.Javascript object>